## Tool Calling or Function Calling
- LLM Automatically calls the function based on the query
- Function parameters are automatically passed to the function
- It is one of the essential requirements of the Agent
- Not all LLM supports tool calling.

- Chat models that support tool calling features implement a .bind_tools() method for passing tool schemas to the model.

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [10]:
os.chdir("..")

In [11]:
ls

app/       data/      notebooks/


In [3]:
#from langchain_ollama import ChatOllama 
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini",temperature=0)
#llm = ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434') #llama3.2:3b-instruct-fp16 phi4:latest
#llm = ChatOllama(model='llama3.2:3b-instruct-fp16', base_url='http://localhost:11434')
#llm.invoke('hi')

### Custom Tools

In [4]:
import yfinance as yf

# Selecciona el símbolo de la acción de interés, por ejemplo, NVIDIA (NVDA)
symbol = "NVDA"
company = yf.Ticker(symbol)

# Usar 'info' para obtener el precio de mercado actual
market_price_info = company.info
print(f"Precio actual de {symbol}: {market_price_info['currentPrice']}")

# Alternativamente, usa 'history' para obtener el precio más reciente
market_data = company.history(period="15d")
closing_price = market_data['Close'].pct_change().dropna()
print(f"Retornos precio de cierre {symbol} ultimos {len(closing_price)} dias: {closing_price}")

Precio actual de NVDA: 111.01
Retornos precio de cierre NVDA ultimos 14 dias: Date
2025-04-07 00:00:00-04:00    0.035309
2025-04-08 00:00:00-04:00   -0.013724
2025-04-09 00:00:00-04:00    0.187227
2025-04-10 00:00:00-04:00   -0.059127
2025-04-11 00:00:00-04:00    0.031235
2025-04-14 00:00:00-04:00   -0.001983
2025-04-15 00:00:00-04:00    0.013459
2025-04-16 00:00:00-04:00   -0.068717
2025-04-17 00:00:00-04:00   -0.028711
2025-04-21 00:00:00-04:00   -0.045128
2025-04-22 00:00:00-04:00    0.020431
2025-04-23 00:00:00-04:00    0.038629
2025-04-24 00:00:00-04:00    0.036218
2025-04-25 00:00:00-04:00    0.043033
Name: Close, dtype: float64


In [12]:
from app.tools.custom_yahoo_finance_tools import YahooFinanceNewsSummaryTool

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [14]:
yf_news_summary_tool =YahooFinanceNewsSummaryTool()

In [15]:
#yf_news_summary_tool.invoke(symbol)
print(yf_news_summary_tool.invoke(symbol))

Is NVIDIA Corporation (NVDA) the Best Stock to Invest in for Long Term?
We recently published a list of 10 Best Stocks to Invest in for Long Term. In this article, we are going to take a look at where NVIDIA Corporation (NASDAQ:NVDA) stands against other best stocks to invest in for long term. The markets have had two monster years with the S&P 500 index surging 24.73% […]

Why a Cautious, Longtime Investor Is Rather Bullish on NVDA
Harvest Portfolio Management Chief Investment Officer​​ Paul Meeks is cautious on the Mag 7 overall, suggesting that investors not buy any of the names until after they report their first-quarter earnings. Moreover, he does not believe that any of the companies will have much confidence in their outlooks, and he does not expect AI to […]


## Calling In-Built Tool 
### Yahoo Finance News

In [ ]:
from langchain.agents import AgentType, initialize_agent
#from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool

In [10]:
# %pip install --upgrade --quiet  yfinance

In [115]:
import yfinance

In [117]:
query = "NVDA"

### DuckDuckGo Search
- There are so many other paid options are also available like Tavily, Google, Bing, etc.

In [11]:
# https://python.langchain.com/docs/integrations/tools/

# !pip install -qU duckduckgo-search wikipedia xmltodict tavily-python

In [97]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("What is today's stock market news?")

/Users/andrestrepo/Documents/repos_personal/Langchain-and-Ollama/.venv/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: lxml is not installed. Using backend='api'.
  ddgs_gen = ddgs.text(


"The stock market is often discussed as a solved problem — much like baseball, with its three true outcomes — but if the known correct action is, as mentioned above, to buy the basket and do ... Stock Market News And Analysis. The analysis you'll find in the Stock Market Today is based on over 130 years of market history and a detailed study of every top-performing stock since the 1880s. In the bond market, Treasury yields were mixed. The yield on the 10-year Treasury went from 4.28% shortly before the release of the retail sales report to nearly 4.33% immediately afterward. It then pulled back to 4.30%, down from its 4.31% level late Friday. In stock markets abroad, indexes rose across much of Europe and Asia. On Monday the stock market shrugged off escalating trade tensions, with all three major averages rallying. Meanwhile, West Texas Intermediate crude ( CL=F ) and Brent futures ( BZ=F ) gained more ... The economic data didn't get any better on Monday, but the stock market was ris

### Tavily Search

In [100]:
from langchain_community.tools import TavilySearchResults

search = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
)

In [101]:
question = "what is today's stock market news?"
search.invoke(question)

[{'url': 'https://finance.yahoo.com/topic/stock-market-news/',
  'content': "### It's 'economic blackout' day as campaign encourages Americans not to spend for 24 hours A retail boycott is hitting major US businesses Friday as an online campaign calls for Americans to spend nothing at places such as Walmart, Target, Amazon and McDonald’s.\n\n### Stock market today: Dow, S&P 500, Nasdaq rally to cap volatile week, losing month Investors are waiting for the release of the Federal Reserve's preferred inflation gauge as they eye Trump’s latest trade threats. [...] ### Why Tesla's stock has given up nearly all of its Trump election gains Tesla stock has nearly given up all its gains since President Trump’s election win, as Elon Musk’s forays with politics have investors wondering where the CEO’s attention lies.\n\n### Stock market today: Nasdaq, S&P 500 sell-off as Nvidia sinks 8%, Trump reiterates tariff plans An upbeat tone is seeping into markets as investors dig into Nvidia's quarterly 

### Wikipedia Search

In [103]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

question = "What is etf?"

print(wikipedia.invoke(question))

Page: Exchange-traded fund
Summary: An exchange-traded fund (ETF) is a type of investment fund that is also an exchange-traded product, i.e., it is traded on stock exchanges. ETFs own financial assets such as stocks, bonds, currencies, debts, futures contracts, and/or commodities such as gold bars. Many ETFs provide some level of diversification compared to owning an individual stock.
An ETF divides ownership of itself into shares that are held by shareholders. Depending on the country, the legal structure of an ETF can be a corporation, trust, open-end management investment company, or unit investment trust. Shareholders indirectly own the assets of the fund and are entitled to a share of the profits, such as interest or dividends, and would be entitled to any residual value if the fund undergoes liquidation. They also receive annual reports. An ETF generally operates with an arbitrage mechanism designed to keep it trading close to its net asset value, although deviations can occur.
T

### Tool Calling with LLM

In [77]:
from langchain_core.tools import tool

@tool
def wikipedia_search(query):
    """
    Search wikipedia for general information about trading and investing.
    
    Args:
    query: The search query
    """
    
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    response = wikipedia.invoke(query)
    return response

@tool
def tavily_search(query):
    """
    Search the web for realtime and latest information.
    for examples, news, stock market etc.
    
    Args:
    query: The search query
    """
    
    search = TavilySearchResults(
        max_results=5,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=True,
    )
    response = search.invoke(query)
    return response



@tool
def duck_search(query):
    """
    Search the web for realtime and latest information.
    for examples, news, stock market etc.
    
    Args:
    query: The search query
    """
    
    search = DuckDuckGoSearchRun()

    response = search.invoke(query)
    return response

@tool
def yahoo_finance_news_search(symbol):
    """
    Search financial news from yahoo finance using the company ticker or symbol.
    
    Args:
    symbol: the company ticker
    """
    
    search =YahooFinanceNewsSummaryTool()

    response = search.invoke(symbol)
    return response


In [78]:
tools = [wikipedia_search, duck_search, yahoo_finance_news_search]

list_of_tools = { tool.name: tool for tool in tools }

In [79]:
list_of_tools

{'wikipedia_search': StructuredTool(name='wikipedia_search', description='Search wikipedia for general information about trading and investing.\n\nArgs:\nquery: The search query', args_schema=<class 'langchain_core.utils.pydantic.wikipedia_search'>, func=<function wikipedia_search at 0x13b3c1080>),
 'duck_search': StructuredTool(name='duck_search', description='Search the web for realtime and latest information.\nfor examples, news, stock market etc.\n\nArgs:\nquery: The search query', args_schema=<class 'langchain_core.utils.pydantic.duck_search'>, func=<function duck_search at 0x13b3c0ea0>),
 'yahoo_finance_news_search': StructuredTool(name='yahoo_finance_news_search', description='Search financial news from yahoo finance using the company ticker or symbol.\n\nArgs:\nsymbol: the company ticker', args_schema=<class 'langchain_core.utils.pydantic.yahoo_finance_news_search'>, func=<function yahoo_finance_news_search at 0x13b3c1440>)}

In [80]:
llm_with_tools = llm.bind_tools(tools)

In [81]:
#query = "What is the latest news"
#query = "What is today's stock market news?"
#query = "what is ETF?"
query = "NVDA"

response = llm_with_tools.invoke(query)
print(response.tool_calls)

[{'name': 'yahoo_finance_news_search', 'args': {'symbol': 'NVDA'}, 'id': '6bc0d90c-3422-4d07-ba93-a9fe0e047571', 'type': 'tool_call'}]


### Generate Final Result with Tool Calling

In [82]:
from langchain_core.messages import HumanMessage, AIMessage

In [83]:
query = "NVDA"

messages = [HumanMessage(query)]

tool_calls = llm_with_tools.invoke(messages)

messages.append(tool_calls)

tool_calls = tool_calls.tool_calls


In [84]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

messages.append(ai_msg)

In [85]:
for tool_call in ai_msg.tool_calls:
    print(tool_call)

    name = tool_call['name'].lower()

    selected_tool = list_of_tools[name]

    tool_msg = selected_tool.invoke(tool_call)

    messages.append(tool_msg)

{'name': 'yahoo_finance_news_search', 'args': {'symbol': 'NVDA'}, 'id': '0c65661a-2c77-4d8b-ba5e-9b7c3a8fe298', 'type': 'tool_call'}


In [86]:
messages

[HumanMessage(content='NVDA', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b-instruct-fp16', 'created_at': '2025-03-18T05:30:31.131874Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'yahoo_finance_news_search', 'arguments': {'symbol': 'NVDA'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 1292201917, 'load_duration': 31435333, 'prompt_eval_count': 294, 'prompt_eval_duration': 219000000, 'eval_count': 21, 'eval_duration': 1041000000}, id='run-994393e6-f856-4f7a-b65d-f3063f7d0daa-0', tool_calls=[{'name': 'yahoo_finance_news_search', 'args': {'symbol': 'NVDA'}, 'id': '0c65661a-2c77-4d8b-ba5e-9b7c3a8fe298', 'type': 'tool_call'}], usage_metadata={'input_tokens': 294, 'output_tokens': 21, 'total_tokens': 315}),
 ToolMessage(content='Fed meeting, US housing starts, Nvidia GTC: What to Watch\nAsking for a Trend host Julie Hyman previews the biggest stories a

In [87]:
response = llm_with_tools.invoke(messages)
print(response.content)

Here's a formatted answer based on the tool call response for NVDA:

"NVIDIA Corporation (NVDA) is a leading manufacturer of graphics processing units (GPUs) and high-performance computing hardware. The company's stock has been impacted by recent market fluctuations, but some analysts see it as a strong buy opportunity.

According to Jim Cramer, CEO Jensen Huang will be delivering his keynote speech at the 2025 GPU Technology Conference, which could provide insight into the company's future plans and strategies.

In terms of technical analysis, Rich Ross from EvercoreISI suggests that some of NVIDIA's top momentum names, including NVDA, have become deeply oversold. However, it's unclear when stability will return to these stocks, and investors should exercise caution in their trading decisions.

Overall, NVDA remains a key player in the technology sector, with its strong brand and diverse product portfolio positioning it for long-term success."
